In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"D:\Ultimate Programming\Data Bases\Project Dataset\price of a Football player.csv")
df.head(2)

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0


In [3]:
df['market_value'] = df.pop('market_value')

q1 = df['page_views'].quantile(0.25)
q3 = df['page_views'].quantile(0.75)
iqr = q3 - q1
min_range = q1 - iqr*1.5
max_range = q3 + iqr*1.5

df = df[df['page_views']<max_range]

In [4]:
# Encoding the dataset
from sklearn.preprocessing import LabelEncoder
name = df.drop('name', axis=1, inplace=True)
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

In [5]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
for col in df.select_dtypes(include=['int64', 'float64']).columns :
    df[col] = ss.fit_transform(df[[col]])

In [6]:
df['region'] = df['region'].fillna(df['region'].mean())

In [7]:
x = df.iloc[:, :-1]
y = df['market_value']

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [13]:
import random
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

pop_size = 50
num_generations = 30
mutation_rate = 0.1


population = [np.random.rand(x_train.shape[1]) for _ in range(pop_size)]

def fitness(weights):
    weighted_x_train = x_train * weights
    weighted_x_test = x_test * weights
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(weighted_x_train, y_train)
    y_pred = model.predict(weighted_x_test)
    return -mean_squared_error(y_test, y_pred)

def crossover(parent1, parent2):
    point = random.randint(0, len(parent1) - 1)
    return np.concatenate((parent1[:point], parent2[point:]))

def mutate(weights):
    if random.random() < mutation_rate:
        idx = random.randint(0, len(weights) - 1)
        weights[idx] = random.random()
    return weights

for _ in range(num_generations):
    population = sorted(population, key=lambda x: fitness(x), reverse=True)
    new_population = population[:10]  # Select top 10
    for _ in range(pop_size - 10):
        p1, p2 = random.sample(new_population, 2)
        offspring = mutate(crossover(p1, p2))
        new_population.append(offspring)
    population = new_population

best_weights = population[0]
print("Optimized Weights:", best_weights)

Optimized Weights: [0.17600689 0.22100595 0.22417105 0.02696349 0.67028463 0.51349374
 0.35835546 0.82115326 0.62446584 0.16203144 0.48524767 0.92420456
 0.56414426 0.452387   0.41121152]
